## Importação de Dados
- Visão Geral dos Dados
    * Diagrama
* Fontes
    - Postgres
    
- Group By
- Pivot
- Functions
- Exclude

In [ ]:
import duckdb
con = duckdb.connect() 
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [ ]:
%%sql 

INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=lord_duckdb_postgres user=lord password=postgres123 host=127.0.0.1' AS db (TYPE POSTGRES);

SHOW ALL TABLES;

In [ ]:
%%sql

CALL postgres_execute('db', ' CREATE TABLE  RECOMENDACAO (
            codigo_ocorrencia4 INTEGER,
            recomendacao_numero varchar(2000),
            recomendacao_dia_assinatura DATE null,
            recomendacao_dia_encaminhamento DATE null,
            recomendacao_dia_feedback DATE null,
            recomendacao_status VARCHAR(2000),
            recomendacao_destinatario_sigla VARCHAR(2000),
            recomendacao_destinatario VARCHAR(2000),
            recomendacao_conteudo TEXT
        );');




In [ ]:
!pip install psycopg2 --quiet

In [ ]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)
cursor = conn.cursor()
cursor.execute("delete from recomendacao;")
# Open and read the CSV file
with open('data/aula5/recomendacao.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')
    next(csvreader)  # Skip the header row
    for row in csvreader:
        # Replace "NULL" with None for date fields
        row = [None if value == "NULL" else value for value in row]
        #print(row)
        # Assuming the CSV columns match the table structure
        cursor.execute('''
            INSERT INTO recomendacao (
                codigo_ocorrencia4, recomendacao_numero, recomendacao_dia_assinatura, 
                recomendacao_dia_encaminhamento, recomendacao_dia_feedback, recomendacao_status, 
                recomendacao_destinatario_sigla, recomendacao_destinatario, recomendacao_conteudo
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row)

# Commit and close
conn.commit()
conn.close()

In [ ]:
%%sql 

CALL postgres_execute('db', 'drop table OCORRENCIA; CREATE TABLE IF NOT EXISTS OCORRENCIA (
            codigo_ocorrencia1 INTEGER,
            codigo_ocorrencia4 INTEGER,
            codigo_ocorrencia2 INTEGER,
            codigo_ocorrencia3 INTEGER,
            codigo_ocorrencia INTEGER,
            ocorrencia_classificacao VARCHAR(255),
            ocorrencia_longitude double precision,
            ocorrencia_latitude double precision,
            ocorrencia_cidade VARCHAR(255),
            ocorrencia_uf VARCHAR(255),
            ocorrencia_pais VARCHAR(255),
            ocorrencia_aerodromo VARCHAR(255),
            ocorrencia_dia DATE,
            ocorrencia_hora TIME null,
            investigacao_aeronave_liberada varchar(3),
            investigacao_status VARCHAR(255),
            divulgacao_relatorio_numero varchar(255),
            divulgacao_relatorio_publicado varchar(3),
            divulgacao_dia_publicacao DATE null,
            total_recomendacoes INTEGER,
            total_aeronaves_envolvidas varchar(3),
            ocorrencia_saida_pista VARCHAR(255)
        );');



In [ ]:
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)

import re

def replace_dots_after_first(value):
    value = value.replace(',', '.')
    dots = [m.start() for m in re.finditer('\.', value)]
    
    # Se houver mais de um ponto, substitui todos os pontos após o primeiro por uma string vazia
    if len(dots) > 1:
        return value[:(dots[0]+1)]+value[(dots[0]+1):].replace('.', '')
    return value

def formatDate(value):
    if value == 'NULL':
        return None
    arr = value.split('/')
    return arr[2]+'-'+arr[1]+'-'+arr[0]

cursor = conn.cursor()
cursor.execute("delete from ocorrencia;")
# Open and read the CSV file
with open('data/aula5/ocorrencia.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')
    next(csvreader)  # Skip the header row
    for row in csvreader:
        # Replace "NULL" with None for date fields
        #print(row) 
        if(row[6] != 'NULL' and row[6] != '' and '*' not in row[6]):
            row[6] = replace_dots_after_first(row[6]) 
        if(row[7] != 'NULL' and row[7] != '' and '*' not in row[7]): 
            row[7] = replace_dots_after_first(row[7]) 
        else:
            row[6] = None
            row[7] = None    

        row[12] = formatDate(row[12])
        row[18] = formatDate(row[18])
        row[13] = None if row[13] == 'NULL' else row[13]
        row[14] = None if row[14] == 'NULL' else row[14]
        try:
            cursor.execute('''
                INSERT INTO ocorrencia (
                    codigo_ocorrencia1, codigo_ocorrencia4, codigo_ocorrencia2, codigo_ocorrencia3, 
                    codigo_ocorrencia, ocorrencia_classificacao, ocorrencia_longitude, ocorrencia_latitude, 
                    ocorrencia_cidade, ocorrencia_uf, ocorrencia_pais, ocorrencia_aerodromo, 
                    ocorrencia_dia, ocorrencia_hora, investigacao_aeronave_liberada, investigacao_status, 
                    divulgacao_relatorio_numero, divulgacao_relatorio_publicado, divulgacao_dia_publicacao, 
                    total_recomendacoes, total_aeronaves_envolvidas, ocorrencia_saida_pista
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', row)
        except Exception as e:
            print(e)
            break
    
# Commit and close
conn.commit()
conn.close()

In [62]:
%%sql 

CALL postgres_execute('db', 'CREATE TABLE IF NOT EXISTS OCORRENCIA_TIPO (
            codigo_ocorrencia1 INTEGER,
            ocorrencia_tipo VARCHAR(255),
            ocorrencia_tipo_categoria VARCHAR(255),
            taxonomia_tipo_icao VARCHAR(255)
        );');



Running query in 'duckdb'

Success


In [63]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)

cursor = conn.cursor()
cursor.execute("delete from ocorrencia_tipo;")
# Assuming you have a CSV file named 'ocorrencia_tipo.csv' with the appropriate columns
with open('data/aula5/ocorrencia_tipo.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')  # Adjust delimiter if necessary
    next(csvreader)  # Skip the header row if your CSV has one
    for row in csvreader:
        try:
            cursor.execute('''
                INSERT INTO OCORRENCIA_TIPO (
                    codigo_ocorrencia1, ocorrencia_tipo, ocorrencia_tipo_categoria, taxonomia_tipo_icao
                ) VALUES (%s, %s, %s, %s)
            ''', row)
        except Exception as e:
            print(row)
            print(e)
            break
# Commit and close
conn.commit()
conn.close()

In [70]:
%%sql 

CALL postgres_execute('db', 'drop table AERONAVE; CREATE TABLE IF NOT EXISTS AERONAVE (
            codigo_ocorrencia2 INTEGER,
            aeronave_matricula VARCHAR(255),
            aeronave_operador_categoria VARCHAR(255),
            aeronave_tipo_veiculo VARCHAR(255),
            aeronave_fabricante VARCHAR(255),
            aeronave_modelo VARCHAR(255),
            aeronave_tipo_icao VARCHAR(255),
            aeronave_motor_tipo VARCHAR(255),
            aeronave_motor_quantidade varchar(255),
            aeronave_pmd VARCHAR(255),
            aeronave_pmd_categoria VARCHAR(255),
            aeronave_assentos INTEGER null,
            aeronave_ano_fabricacao INTEGER null,
            aeronave_pais_fabricante VARCHAR(255),
            aeronave_pais_registro VARCHAR(255),
            aeronave_registro_categoria VARCHAR(255),
            aeronave_registro_segmento VARCHAR(255),
            aeronave_voo_origem VARCHAR(255),
            aeronave_voo_destino VARCHAR(255),
            aeronave_fase_operacao VARCHAR(255),
            aeronave_tipo_operacao VARCHAR(255),
            aeronave_nivel_dano VARCHAR(255),
            aeronave_fatalidades_total INTEGER
        );');



Running query in 'duckdb'

Success


In [72]:
import csv
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="lord_duckdb_postgres", 
    user="lord", 
    password="postgres123", 
    host="127.0.0.1", 
    port="5432"
)
cursor = conn.cursor()
cursor.execute("delete from aeronave;")
# Assuming you have a CSV file named 'aeronave.csv' with the appropriate columns
with open('data/aula5/aeronave.csv', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=';')  # Adjust delimiter if necessary
    next(csvreader)  # Skip the header row if your CSV has one
    for row in csvreader:
        # Replace "NULL" with None for date fields
        row[11] = None if row[11] == 'NULL' else row[11]
        cursor.execute('''
            INSERT INTO AERONAVE (
                codigo_ocorrencia2, aeronave_matricula, aeronave_operador_categoria, 
                aeronave_tipo_veiculo, aeronave_fabricante, aeronave_modelo, 
                aeronave_tipo_icao, aeronave_motor_tipo, aeronave_motor_quantidade, 
                aeronave_pmd, aeronave_pmd_categoria, aeronave_assentos, 
                aeronave_ano_fabricacao, aeronave_pais_fabricante, aeronave_pais_registro, 
                aeronave_registro_categoria, aeronave_registro_segmento, aeronave_voo_origem, 
                aeronave_voo_destino, aeronave_fase_operacao, aeronave_tipo_operacao, 
                aeronave_nivel_dano, aeronave_fatalidades_total
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row)

# Commit and close
conn.commit()
conn.close()

InvalidTextRepresentation: invalid input syntax for type integer: "NULL"
LINE 11: ...0', 'AT76', '***', '***', '23000', '23000', NULL, 'NULL', 'B...
                                                              ^


In [ ]:
%%sql 

CALL postgres_execute('db', 'CREATE TABLE IF NOT EXISTS FATOR_CONTRIBUINTE (
            codigo_ocorrencia3 INTEGER,
            fator_nome VARCHAR(255),
            fator_aspecto VARCHAR(255),
            fator_condicionante VARCHAR(255),
            fator_area VARCHAR(255)
        );');



## Exportação de Dados
- Visão Geral
- Saídas
    - S3 (minio/aws)
        - Delta Sharing
        - Power BI
        - Trino

    - Embedded Data Analysis 
        - Superset 
        
    - CSV
    - Json

In [ ]:
#Export Data